In [59]:
# Always start with a dataset to train on. Downloading tiny shakespeare.
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# from urllib.request import urlretrieve
# urlretrieve("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt","input.txt")

In [60]:
with open('input.txt', 'r',encoding='utf-8') as f:
    text = f.read()

In [61]:
print("length of database in characters", len(text))

length of database in characters 1115394


In [62]:
# Lets look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [63]:
# these are all the unique characters that occur in this text
chars = sorted(list(set(text))) # sorts the values of the chars into a new set list
vocab_size = len(chars) # 65 total characters
print(''.join(chars)) 
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [64]:
# create mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)} # using the chars dictionary it creates a number for each char and calls it to make it a integer
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("First Citizen: Before we proceed any further, hear me speak"))
print(decode([18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 1, 14, 43, 44, 53, 56, 
              43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 
              46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49]))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 1, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49]
First Citizen: Before we proceed any further, hear me speak


In [65]:
# encode the data and store as torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long) #Stores the encodings of the characters in tensors
print(data.shape, data.dtype) # has all the text file with the return type of integers
print(data[:1000]) # imagine a new array here


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [66]:
# Split data into train and validation sets
n = int(0.9 * len(data)) # Train 90 percent of the values and let the rest be the validation
train_data = data[:n]
val_data = data[n:]

In [67]:
block_size = 8 # do not let the block be more than 8 characters long
train_data[:block_size+1] # then in order to train the data we need one more value to look at to compare the estimation

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [68]:
# Train the transformer with block sizes so that in a size of 9 there are 8 individual targets which can be repeated simultaneously by the gpu
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}") #  All this tells you is that at a certain time the tensor is validating with the target
# The more we repeat this process the more likely we will get a closer probablity of the values

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [69]:
# TODO: Understand what is happening 
torch.manual_seed(1337) 
batch_size = 4 # runs 4 batches at the same time
block_size = 8 # takes in 8 characters at the same time

# Gets a batch of the value of the string
def get_batch(split):
    # Splits the values into train data and val data
    data = train_data if split == 'train' else val_data
    # makes a number for the length of ix
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # makes a stack with the data with the block size using the random number to get the value necessary
    x = torch.stack([data[i:i+block_size] for i in ix])
    # makes a stack of the values one greater than the ones in x
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
    
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
            

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 5

In [70]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [71]:
print(yb)

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [72]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# For reproducibility
torch.manual_seed(1337)

# Bigram language model is a subclass of neural net module in pytorch 
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # makes a embeddding table of size vocab by vocab to get the logits for the next token using a lookup table
        # 24 in the xb table will go to the embedding table and look at the row of values in the embedding table row 24 
        # 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        # idx is the current context of the character in a batch
        # Batch = 4
        # Time =8 
        # Channel = vocab_size
        logits = self.token_embedding_table(idx) # (B,T,C)
        # the logits are the scores for the next character in the sequence 
        # Basically trying to find what comes next from the identity of a single token 

        if targets is None:
            loss = None
        else :
             # Reshape logits for pytorch library
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # A good way to see how good the scores where at predicting we use the loss fucntion
            # how well are we predicting the next character based on the logits
            loss = F.cross_entropy(logits, targets)
            
        # Expecting the loss to be -ln(1/65)
        return logits, loss

    # Getting the indices to pass into the next probability but now this is just looking at the past value
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get predictions with the current indices
            logits, loss = self(idx)
            # Focus only on the last step pluc out the last element in the time dimension 
            logits = logits[:, -1, :] 
            # Convert to probabilities
            probs = F.softmax(logits, dim=-1)
            # use multinomial distribution to sample from the probabilities and get a sample 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # what is generated in idx_next is concatonated ontop of the previous idx to create the (B,T + 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
        

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
# creating a 1,1 tensor containing zero makes the first character 
# get 100 tokens and then runs generate then we take the first dimension idx and turn it into a list and decode it
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100) [0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [73]:
# Optimizing the learning 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [74]:
batch_size = 32
for steps in range(1000):
    # Sample batch of data
    xb, yb = get_batch('train')
    # Evaluate the loss 
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


3.721843719482422


In [75]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500) [0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-bTop-QJe.H?x
JGF&pwst-P sti.hlEsu;w:w a BG:tLhMk,epdhlay'sVzLq--ERwXUzDnq-bn czXxxI&V&Pynnl,s,Ioto!uvixwC-IJXElrgm C-.bcoCPJ
IMphsevhO AL!-K:AIkpre,
rPHEJUzV;P?uN3b?ohoRiBUENoV3B&jumNL;Aik,
xf -IEKROn JSyYWW?n 'ay;:weO'AqVzPyoiBL? seAX3Dot,iy.xyIcf r!!ul-Koi:x pZrAQly'v'a;vEzN
BwowKo'MBqF$PPFb
CjYX3beT,lZ qdda!wfgmJP
DUfNXmnQU mvcv?nlnQF$JUAAywNocd  bGSPyAlprNeQnq-GRSVUP.Ja!IBoDqfI&xJM AXEHV&DKvRS


In [76]:
# Math trick for self-attention weakest form of agregation 
torch.manual_seed(1337)
# We could like to couple the 8 tokens 
# 5th token can get info from the past not the future
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [77]:
# Want x[b,t] = mean_{i<=i} x[b,i]
# X Bag of words averging the words
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        # t the previous time and C the values of the 2d tokens
        xprev = x[b,:t+1]
        # makes the average of the previous values 
        xbow[b,t] = torch.mean(xprev, 0)

In [78]:
# weights array, or how much of every row we want averaged up
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2 = wei @ x # T x T @ B x T x C give B x T x C
torch.allclose(xbow, xbow2)

False

In [79]:
# Version 3 uses softmax
# Use lower trinagular matrix multiplication and normalization to see how much affinity the values have
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
# if tril equals 0 put negative infinity
wei = wei.masked_fill(tril == 0, float('-inf'))
# We normalize the values based on how interesting they find eachother
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [80]:
# Version 4 Self attention for single head 
B,T,C = 4, 8, 32 # Information at each token is 32 dimensional
x = torch.randn(B,T,C)

# each token has a key and a query and then the dot product tells up how they interact
head_size = 16
# matrix multiplication with a weighted matrix 
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x) # B T 16
q = query(x) # B T 16
# Value 
value = nn.Linear(C, head_size, bias=False)

wei = q @ k.transpose(-2, -1)
# Dot product between keys and queries, if the keys and the queries are alligned then we will learn more from the specific token 
# Create channels for each token
tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
# Aggregating the entries themselve to get a greater affinity, wei has to be very diffused as softmax 
# will sharpen toward the max
wei = F.softmax(wei, dim=-1)

# Takes in the value x which has the values
v = value(x)
# out = wei @ x
out = wei @ v

out.shape 


torch.Size([4, 8, 16])

- Attention is a commincation mechanism, can be seen as nodes in a directed graph looking at each other and aggregation information
with a weighted sum from all nodes that point to them, with data-dependant weights.
- There is no notion of space, attention simply acts over a set of vectors, this is why we need to positionally encode tokens
- Each example across batch dimension is of course processed completely independant and never "talk" to each other
- In an "encoder block" just means that you delete the single line that does masking with tril allowing all tokens to communicate.
- Self attention means that the same source x, cross attention means that there is another thing we would like to look at
- "Scalled attention" divides wei by 1/sqrt(head_size). This makes it so when input Q, K are unit variance, wei will be unit variance too and softmax will stay diffuse and not saturate too much.  

In [81]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [5.4949e-01, 4.5051e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.0814e-02, 1.8298e-02, 9.5089e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.7306e-02, 5.9927e-03, 9.6422e-01, 2.4795e-03, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0356e-01, 8.4526e-02, 2.9864e-01, 4.2126e-01, 9.2020e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [5.5813e-02, 2.6597e-01, 6.9153e-02, 8.4581e-02, 2.7192e-01, 2.5256e-01,
         0.0000e+00, 0.0000e+00],
        [1.1987e-02, 3.8042e-04, 9.6417e-01, 3.2126e-03, 5.0234e-03, 1.4995e-02,
         2.3348e-04, 0.0000e+00],
        [3.9488e-02, 2.8300e-01, 1.5332e-02, 7.4047e-02, 5.5218e-02, 4.9383e-01,
         1.6254e-02, 2.2827e-02]], grad_fn=<SelectBackward0>)

In [82]:
torch.manual_seed(1337)
# Can use torch.tril to make a triangular matrix to all us access the previous number while not looking at the previous
# a = torch.ones(3,3)
a = torch.tril(torch.ones(3,3)) 
a = a / torch.sum(a,1,keepdim=True) # Average of the first two rows
b = torch.randint(0,10,(3,2)).float()
# matrix multiplication
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])


In [83]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [84]:
k.var()

tensor(0.9797)

In [85]:
q.var()

tensor(1.0390)

In [86]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [94]:
# Normalizes the rows  
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        # self.momentum = momentum
        # self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # Buffers trained with a running momentum update
        # self.running_mean = torch.zeros(dim)
        # self.running_var = torch.ones(dim)
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        # Do not need to maintain buffers
        # if self.training:
        #     with torch.no_grad():
        #         self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        #         self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]
torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32,100)
x = module(x)
x.shape

torch.Size([32, 100])

In [92]:
x[:, 0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))